# Homework #8 for ML zoomcamp course

## Data loading

In [1]:
#!wget https://github.com/alexeygrigorev/dino-or-dragon/releases/download/data/dino-dragon.zip
#!unzip dino-dragon.zip

In [2]:
!ls

dino-dragon.zip  Homework.ipynb  test  train


# Create model

In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Conv2D,MaxPooling2D,Flatten, Dense
from tensorflow.keras.optimizers import SGD

2022-11-20 02:37:48.487739: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
def create_model(input_shape=150):
    model = Sequential()
    model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(150,150,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    return model


In [5]:
model = create_model()

2022-11-20 02:37:50.394838: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-20 02:37:50.402772: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-20 02:37:50.403560: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-20 02:37:50.404809: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
____________________________________________

In [7]:
model.compile(optimizer=SGD(learning_rate=0.002, momentum=0.8), loss="binary_crossentropy", metrics=["accuracy"])

# Train a model

In [8]:
! pip install scipy

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import scipy

In [10]:
idg = ImageDataGenerator(rescale=1./255)

In [11]:
train_gen = idg.flow_from_directory("train", class_mode="binary", shuffle=True, batch_size=20, target_size=(150, 150))
test_gen = idg.flow_from_directory("test", class_mode="binary", shuffle=True, batch_size=20, target_size=(150, 150))

Found 1596 images belonging to 2 classes.
Found 394 images belonging to 2 classes.


In [12]:
history = model.fit(train_gen, epochs=10, validation_data=test_gen)

Epoch 1/10


2022-11-20 02:37:56.236437: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-11-20 02:37:58.485055: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-20 02:37:58.485971: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-20 02:37:58.486012: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-11-20 02:37:58.487092: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-20 02:37:58.487189: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


80/80 [==============================] - 17s 127ms/step - loss: 0.6199 - accuracy: 0.6366 - val_loss: 0.5053 - val_accuracy: 0.8325
Epoch 2/10
80/80 [==============================] - 9s 113ms/step - loss: 0.4641 - accuracy: 0.8089 - val_loss: 0.4158 - val_accuracy: 0.8274
Epoch 3/10
80/80 [==============================] - 9s 110ms/step - loss: 0.3751 - accuracy: 0.8503 - val_loss: 0.3610 - val_accuracy: 0.8452
Epoch 4/10
80/80 [==============================] - 9s 111ms/step - loss: 0.3310 - accuracy: 0.8678 - val_loss: 0.4804 - val_accuracy: 0.7690
Epoch 5/10
80/80 [==============================] - 9s 110ms/step - loss: 0.2846 - accuracy: 0.8891 - val_loss: 0.3484 - val_accuracy: 0.8477
Epoch 6/10
80/80 [==============================] - 9s 112ms/step - loss: 0.2690 - accuracy: 0.8991 - val_loss: 0.3037 - val_accuracy: 0.8706
Epoch 7/10
80/80 [==============================] - 9s 109ms/step - loss: 0.2267 - accuracy: 0.9204 - val_loss: 0.2850 - val_accuracy: 0.8731
Epoch 8/10
80/80

## Find answers to homework's questions

In [13]:
np.median(history.history["accuracy"])

0.8941102921962738

In [14]:
np.std(history.history["loss"])

0.1352763885654728

# Add augmentations

In [15]:
from tensorflow.keras.callbacks import TensorBoard

log_dir = "logs/fit"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [16]:
idg_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_gen = idg_aug.flow_from_directory("train", class_mode="binary", shuffle=True, batch_size=20, target_size=(150, 150))

Found 1596 images belonging to 2 classes.


In [17]:
history2 = model.fit(train_gen, epochs=10, validation_data=test_gen, callbacks=[tensorboard_callback])

Epoch 1/10
80/80 [==============================] - 18s 222ms/step - loss: 0.4428 - accuracy: 0.7863 - val_loss: 0.2799 - val_accuracy: 0.8782
Epoch 2/10
80/80 [==============================] - 17s 209ms/step - loss: 0.4226 - accuracy: 0.8120 - val_loss: 0.4463 - val_accuracy: 0.7893
Epoch 3/10
80/80 [==============================] - 17s 209ms/step - loss: 0.3990 - accuracy: 0.8164 - val_loss: 0.3873 - val_accuracy: 0.8376
Epoch 4/10
80/80 [==============================] - 17s 209ms/step - loss: 0.3812 - accuracy: 0.8352 - val_loss: 0.3417 - val_accuracy: 0.8477
Epoch 5/10
80/80 [==============================] - 17s 209ms/step - loss: 0.3960 - accuracy: 0.8271 - val_loss: 0.4633 - val_accuracy: 0.8020
Epoch 6/10
80/80 [==============================] - 17s 210ms/step - loss: 0.3728 - accuracy: 0.8421 - val_loss: 0.3837 - val_accuracy: 0.8299
Epoch 7/10
80/80 [==============================] - 17s 208ms/step - loss: 0.3708 - accuracy: 0.8396 - val_loss: 0.2898 - val_accuracy: 0.8756

In [18]:
history2.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

# Answers to questions 5 and 6 (choose the closest values)

In [19]:
np.mean(history2.history["val_loss"])

0.38589749932289125

In [20]:
np.mean(history2.history["val_accuracy"][5:])

0.8340101480484009